# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902013


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:06<01:35,  3.30s/it]

Rendering models:  10%|▉         | 3/31 [00:07<01:16,  2.73s/it]

Rendering models:  16%|█▌        | 5/31 [00:08<00:51,  1.97s/it]

Rendering models:  19%|█▉        | 6/31 [00:08<00:38,  1.53s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:17<00:01,  2.96it/s]

Rendering models:  94%|█████████▎| 29/31 [00:17<00:00,  2.86it/s]

Rendering models:  97%|█████████▋| 30/31 [00:17<00:00,  3.03it/s]

Rendering models: 100%|██████████| 31/31 [00:18<00:00,  2.27it/s]

equidad1                              0.061371
taisribeiro                           0.003672
Rubru                                 0.000327
not-logged-in-e5409037b2fb11f8e397    0.006296
smilingsloth.                         0.049082
xperry9702                            0.000873
not-logged-in-c0884513393674cd89f6    0.005036
Nauter                                0.001435
tingard                               0.000419
becca1029                             0.000645
FeatheredSpinoGamer                   0.001704
Marra                                 0.000954
clars915                              0.000291
malianaumchik                         0.131788
qbright                               0.000587
JoseA.G.v                             0.001887
clairedeu                             0.000497
acapirala                             0.004296
tominas                               0.000259
bigdeath                              0.000499
not-logged-in-fcfcbc88feb1aa0ed0a7    0.000853
Lavadude     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())